# Match BGBM Collectors to Wikidata Items Using *Cosine Similarity*, `eventDate` Involved

In this example we add `eventDate` of the source data, when the sample/occurrence was collected, to have a time reference, when the collector should have been  alive.

Basically we …

- match of `canonical_string` of WikiData to `canonical_string` of the source collectors (abbreviated names and full names, if given), and
- parse collector source names beforehand to get individual names out of name lists in the source data, we have used <https://libraries.io/rubygems/dwc_agent>, and in general we
- follow the example of Niels Klazenga <https://github.com/nielsklazenga/avh-collectors/blob/master/cosine_similarity.ipynb>
- write the output to provide a DarwinCore attribution structure (for `verbatimName` we would need the `source_data` name(s))

Technical Notes — Review Code perhaps:
- TODO review score calculation of the matching of relating eventData with range of yob, yod
- TODO review DwC agent output, keep at this time custom columns for filter-sort-evaluation convenience
- (NN ⇌ Cosine) refactor relation: wd_matchtest ⇌ wikidata_unique (replaced wd_matchtest → wikidata_unique)

### Load Wikidata Data Set

Construct data using Jupyter Notebook [create_wikidata_datasets_botanists.ipynb](./create_wikidata_datasets_botanists.ipynb)

Out of the Wikidata items data set we create a data frame with unique canonical name strings and their counts.

In [1]:
import pandas as pd
import pprint, time, os

wikidata = pd.read_csv(
    "data/wikidata_persons_botanists_20230703_1352.csv", 
    index_col=0, low_memory=False,
    dtype={
        'yob':'Int32',
        'yod':'Int32',
        'wyb':'Int32',
        'wye':'Int32'
    }    
)
# # # # # # # # # # # # # # # # 
# def convert_to_datetime(year):
#     year = int(year)
#     days = int((year - int(year)) * 365.25)
#     base_date = pd.to_datetime(f'{year}-01-01', errors="ignore")
#     # ,format='%Y-%m-%d', errors='coerce'
#     # base_date = pd.Period(year, freq='D')
#     return base_date + pd.DateOffset(days=days)
# 
# def convert_to_datetime(year):
#     base_date = pd.to_datetime(f'{year}-01-01', errors="ignore")
# print(wikidata.dtypes)

# wikidata['yob_converted'] = wikidata['yob'].apply(convert_to_datetime)
# wikidata['yod_converted'] = wikidata['yod'].apply(lambda x: pd.Period(x, freq='Y')) # Given date string "-286" not likely a datetime
# # # # # # # # # # # # # # # # 

pprint.pprint(wikidata.columns)
display(wikidata.head())

Index(['item', 'itemLabel', 'surname', 'initials', 'canonical_string',
       'canonical_string_fullname', 'orcid', 'viaf', 'isni', 'harv', 'ipni',
       'abbr', 'bionomia_id', 'yob', 'yod', 'wyb', 'wye', 'wikidata_link',
       'orcid_link', 'harv_link', 'ipni_link', 'bionomia_link'],
      dtype='object')


,item,itemLabel,surname,initials,canonical_string,canonical_string_fullname,orcid,viaf,isni,harv,...,bionomia_id,yob,yod,wyb,wye,wikidata_link,orcid_link,harv_link,ipni_link,bionomia_link
0,http://www.wikidata.org/entity/Q66612,Friedrich August Marschall von Bieberstein,Bieberstein,F.A.M.v.,"Bieberstein, F.A.M.v.","Bieberstein, Friedrich August Marschall von",NaN,43340073,0000 0001 1630 5464,1373,...,Q66612,1768,1826,<NA>,<NA>,http://www.wikidata.org/wiki/Q66612,NaN,https://kiki.huh.harvard.edu/databases/botanis...,https://www.ipni.org/a/6129-1,https://bionomia.net/Q66612
1,http://www.wikidata.org/entity/Q66934,Hans Hermann Behr,Behr,H.H.,"Behr, H.H.","Behr, Hans Hermann",NaN,20328622,0000 0001 1604 8680,42741,...,Q66934,1818,1904,<NA>,<NA>,http://www.wikidata.org/wiki/Q66934,NaN,https://kiki.huh.harvard.edu/databases/botanis...,https://www.ipni.org/a/619-1,https://bionomia.net/Q66934
2,http://www.wikidata.org/entity/Q66661,Jacob Christian Schäffer,Schäffer,J.C.,"Schäffer, J.C.","Schäffer, Jacob Christian",NaN,47016953,0000 0000 8343 3899,1101,...,NaN,1718,1790,<NA>,<NA>,http://www.wikidata.org/wiki/Q66661,NaN,https://kiki.huh.harvard.edu/databases/botanis...,https://www.ipni.org/a/12818-1,NaN
3,http://www.wikidata.org/entity/Q67003,Johann Friedrich Klotzsch,Klotzsch,J.F.,"Klotzsch, J.F.","Klotzsch, Johann Friedrich",NaN,20426762,0000 0001 1749 2732,135,...,Q67003,1805,1860,<NA>,<NA>,http://www.wikidata.org/wiki/Q67003,NaN,https://kiki.huh.harvard.edu/databases/botanis...,https://www.ipni.org/a/4855-1,https://bionomia.net/Q67003
4,http://www.wikidata.org/entity/Q66322,Franz Anton Menge,Menge,F.A.,"Menge, F.A.","Menge, Franz Anton",NaN,59847236,0000 0001 1653 0899,73782,...,NaN,1808,1880,<NA>,<NA>,http://www.wikidata.org/wiki/Q66322,NaN,https://kiki.huh.harvard.edu/databases/botanis...,https://www.ipni.org/a/23266-1,NaN


In [2]:
# compile data having only unique canonical strings
# group by canonical name/string, count douplicated names
wd_matchtest = wikidata.groupby('canonical_string').agg({'item': ['count']}).reset_index()
wd_matchtest_fullnames = wikidata.groupby('canonical_string_fullname').agg({'item': ['count']}).reset_index()

display(wd_matchtest)
display(wd_matchtest_fullnames)


,canonical_string,item
,,count
0,"(-Walraevens), O.H.",1
1,"(1835-1906), G.A.F.E.",1
2,"(1873-1926), S.S.",1
3,"(1888–1973), G.A.",1
4,"(1904-1990), J.J.",1
...,...,...
61479,"Șerbanescu, I.",1
61480,"Ștefureac, T.",1
61481,"Țopa, E.",1


,canonical_string_fullname,item
,,count
0,"(-Walraevens), O Heylen",1
1,"(1835-1906), Gustav Adolf Ferdinand Eichler",1
2,"(1873-1926), Søren Sørensen",1
3,"(1888–1973), Georges André",1
4,"(1904-1990), Johannes Johannessen",1
...,...,...
63605,"Șerbanescu, Ioan",1
63606,"Ștefureac, Traian",1
63607,"Țopa, Emilian",1


## Load Collectors Data Set

**Data sources:**

- option 1: Jupyter Notebook for [create_bgbm_gbif-occurrence_collectors_dataset.ipynb](./create_bgbm_gbif-occurrence_collectors_dataset.ipynb)
- option 2: Jupyter Notebook for `create_bgbm_botanypilot_collectors_dataset.ipynb` from SPARQL (not in this official documentation yet)

Then parse collector names to get single, separate collector names using `dwcagent`, use ruby gem package available at  <https://rubygems.org/gems/dwc_agent>:

- see [./bin/README.md](bin/README.md) to use ruby script `./bin/agent_parse4tsv.rb` for parsing text lines like `"Abbe,L.B., Abbe,E.C., Smitinand,T. & Rollet,B."`

Technical notes:

- the corresponding objects, variable names of Nils’ python code were:
```
refactor df_avh = → = collectors
refactor df_avh['label'] = → = collectors['canonical_string_collector_parsed']
…
```

In [3]:
# unique names parsed already by ruby gem package: dwcagent

# collectors = pd.read_csv("data/bgbm_collectors_20230510_1429_single-line_parsed_unique_names.tab", sep="\t")
collectors = pd.read_csv("data/VHde_doi-10.15468-dl.tued2e/occurrence_recordedBy_eventDate_occurrenceIDs_20230823_parsed.tsv", sep="\t", low_memory=False)

collectors.dropna(subset=['family'], inplace=True) # remove where family was NA, e.g. from originally «??» aso.

print("modify time using pd.Periode(…) to get it work also on very old dates...")
for col in ['eventDate_mean', 'eventDate_min', 'eventDate_max']:
    print("- convert", col, "to pd.Period(...) in collectors")
    collectors[col] = collectors[col].apply(
        lambda x: pd.Period(
            x, freq='S' if col.lower().endswith('mean') else 'D' # Seconds or Day level
        )
    ) # D=day level
    # see https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#timeseries-period-aliases
print("modifing done.")

collectors.sort_values(by=['family', 'given','occurrenceID_firstsample'], inplace=True)
collectors

modify time using pd.Periode(…) to get it work also on very old dates...
- convert eventDate_mean to pd.Period(...) in collectors
- convert eventDate_min to pd.Period(...) in collectors
- convert eventDate_max to pd.Period(...) in collectors
modifing done.


,family,given,suffix,particle,dropping_particle,nick,appellation,title,source_data,parsed_names,cleaned_names,occurrenceID_count,occurrenceID_firstsample,eventDate_mean,eventDate_min,eventDate_max
2,A. Cano,E.,NaN,NaN,NaN,NaN,NaN,NaN,"A. Cano,E.",parsed:E. A. Cano,cleaned:E. A. Cano,1,https://herbarium.bgbm.org/object/B100699397,2008-06-05 00:00:00,2008-06-05,2008-06-05
31563,Aaiki,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Murashige & Aaiki,parsed:Murashige<SEP>Aaiki,cleaned:Murashige<SEP>Aaiki,1,https://herbarium.bgbm.org/object/B101149305,1949-03-02 00:00:00,1949-03-02,1949-03-02
5,Aaronsohn,A.,NaN,NaN,NaN,NaN,NaN,NaN,"Aaronsohn,A.",parsed:A. Aaronsohn,cleaned:A. Aaronsohn,1,https://herbarium.bgbm.org/object/B100379341,1908-06-20 00:00:00,1908-06-20,1908-06-20
22458,Abaouz,A.,NaN,NaN,NaN,NaN,NaN,NaN,"Jury,S.L., Abaouz,A. ,Lafkih,M.Ait & Griffiths...",parsed:S.L. Jury<SEP>A. Abaouz<SEP>M.Ait Lafki...,cleaned:S.L. Jury<SEP>A. Abaouz<SEP>M. Ait Laf...,3,https://herbarium.bgbm.org/object/B100217620,1997-07-10 16:00:00,1997-07-05,1997-07-14
22462,Abaouz,A.,NaN,NaN,NaN,NaN,NaN,NaN,"Jury,S.L., Abaouz,A., Ait Lafkih,M. & Griffith...",parsed:S.L. Jury<SEP>A. Abaouz<SEP>M. Ait Lafk...,cleaned:S.L. Jury<SEP>A. Abaouz<SEP>M. Ait Laf...,2,https://herbarium.bgbm.org/object/B100326682,1997-07-09 12:00:00,1997-07-07,1997-07-12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54218,Ždanova,O.,NaN,NaN,NaN,NaN,NaN,NaN,"Ždanova,O.",parsed:O. Ždanova,cleaned:O. Ždanova,5,https://herbarium.bgbm.org/object/B100263330,1988-06-17 14:24:00,1986-08-18,1989-09-10
27655,Ždanova,O.,NaN,NaN,NaN,NaN,NaN,NaN,"Lomonosova,M., Ždanova,O. & Šaulo,D.",parsed:M. Lomonosova<SEP>O. Ždanova<SEP>D. Šaulo,cleaned:M. Lomonosova<SEP>O. Ždanova<SEP>D. Šaulo,1,https://herbarium.bgbm.org/object/B100263331,1989-09-03 00:00:00,1989-09-03,1989-09-03
54219,Žíla,V.,NaN,NaN,NaN,NaN,NaN,NaN,"Žíla,V.",parsed:V. Žíla,cleaned:V. Žíla,3,https://herbarium.bgbm.org/object/B100009590,2003-05-17 08:00:00,2000-08-29,2004-09-24
54220,Волкова,Е.,NaN,NaN,NaN,NaN,NaN,NaN,Волкова Е.,parsed:Е. Волкова,cleaned:Е. Волкова,1,https://herbarium.bgbm.org/object/B100530714,1951-07-18 00:00:00,1951-07-18,1951-07-18


### Check Composition of Parsed Collector Data

In [4]:
# TODO review code of abbreviated names and full name matching
criterion_fullnames = collectors.given.str.contains('^\w{3,}', na=False)
print("Show collecors given name has (propably) a full name (%s records) …" % len(collectors[criterion_fullnames].index))
collectors[criterion_fullnames]

Show collecors given name has (propably) a full name (594 records) …


,family,given,suffix,particle,dropping_particle,nick,appellation,title,source_data,parsed_names,cleaned_names,occurrenceID_count,occurrenceID_firstsample,eventDate_mean,eventDate_min,eventDate_max
2022,Abdallah,Raffael,NaN,NaN,NaN,NaN,NaN,NaN,"Bally,P.R.O., Abdallah, Raffael & Reichstein, T.",parsed:P.R.O. Bally<SEP>Raffael Abdallah<SEP>T...,cleaned:P.R.O. Bally<SEP>Raffael Abdallah<SEP>...,1,https://herbarium.bgbm.org/object/B200125981,1969-11-18 00:00:00,1969-11-18,1969-11-18
22541,Abdul,Kadir Bin,NaN,NaN,NaN,NaN,NaN,NaN,Kadir Bin Abdul,parsed:Kadir Bin Abdul,cleaned:Kadir Bin Abdul,1,https://herbarium.bgbm.org/object/B100184021,1950-03-20 00:00:00,1950-03-20,1950-03-20
17232,Abreu,Guilherme,NaN,de,NaN,NaN,NaN,NaN,Guilherme de Abreu (no. 103),parsed:Guilherme de Abreu,cleaned:Guilherme de Abreu,1,http://id.snsb.info/snsb/collection/22086/3086...,1938-08-24 00:00:00,1938-08-24,1938-08-24
15082,Adá,García,NaN,NaN,NaN,NaN,NaN,NaN,"García Adá, Luceño, Rico,E., Romero,T. & Varga...",parsed:García Adá<SEP>Rico Luceño<SEP>E. Romer...,cleaned:García Adá<SEP>Rico Luceño<SEP>E. Rome...,1,https://herbarium.bgbm.org/object/B100296455,1987-06-19 00:00:00,1987-06-19,1987-06-19
399,Ahagen,Schiers,NaN,NaN,NaN,NaN,NaN,NaN,"Ahagen, Schiers,C. & al.",parsed:Schiers Ahagen<SEP>C.<SEP>Al,cleaned:Schiers Ahagen<SEP>C.<SEP>Al,1,https://herbarium.bgbm.org/object/B100194646,1987-07-03 00:00:00,1987-07-03,1987-07-03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53493,Zernig,Kurt,NaN,NaN,NaN,NaN,NaN,NaN,"Zernig,Kurt",parsed:Kurt Zernig,cleaned:Kurt Zernig,3,https://herbarium.bgbm.org/object/B101139614,2019-06-29 08:00:00,2019-05-07,2019-09-28
23469,Zhuo,Zhou,NaN,NaN,NaN,NaN,NaN,NaN,"Kilian,N., Raab-Straube,E. von, Zhang Jianwen,...",parsed:N. Kilian<SEP>E.von Raab-Straube<SEP>Zh...,cleaned:N. Kilian<SEP>E. von Raab-Straube<SEP>...,63,https://herbarium.bgbm.org/object/B100517202,2012-08-26 04:57:08,2012-08-17,2012-09-02
53527,Zickendrath,Ernst,NaN,NaN,NaN,NaN,NaN,NaN,"Zickendrath,Ernst",parsed:Ernst Zickendrath,cleaned:Ernst Zickendrath,3,https://je.jacq.org/JE04006629,1896-11-27 08:00:00,1894-04-03,1898-09-20
53528,Zickendrath,Ernst,NaN,NaN,NaN,NaN,NaN,NaN,"Zickendrath,Ernst & Heyden,K.L.",parsed:Ernst Zickendrath<SEP>K.L. Heyden,cleaned:Ernst Zickendrath<SEP>K.L. Heyden,1,https://je.jacq.org/JE04007139,1900-09-01 00:00:00,1900-09-01,1900-09-01


In [5]:
# check the name-parsed columns if they are empty or need to be considerd as data for matching or not
for parsed_name_part in ["particle", "suffix", "dropping_particle", "appellation"]:
    test_collectors = collectors.loc[(collectors[parsed_name_part].isna() == False)]
    print("\n----------------------------------------\nshow names with **%s** found %s records:\n" % (parsed_name_part, len(test_collectors.index)))
    display(test_collectors.head().get(["family", "given", "suffix", "particle", "dropping_particle", "nick", "appellation", "title"]))    


----------------------------------------
show names with **particle** found 603 records:



,family,given,suffix,particle,dropping_particle,nick,appellation,title
17232,Abreu,Guilherme,NaN,de,NaN,NaN,NaN,NaN
347,Aghababyan,M.,NaN,von,NaN,NaN,NaN,NaN
3480,Aguilar,M.L.,NaN,Reyna de,NaN,NaN,NaN,NaN
49516,Aguilar,M.L.,NaN,Reyna de,NaN,NaN,NaN,NaN
13616,Aguilar,M.L.,NaN,Reyna de,NaN,NaN,NaN,NaN



----------------------------------------
show names with **suffix** found 14 records:



,family,given,suffix,particle,dropping_particle,nick,appellation,title
13966,August,Friedrich,II.,NaN,NaN,NaN,NaN,NaN
47798,Dogma,I.J.,Jr.,NaN,NaN,NaN,NaN,NaN
13752,Forsyth,W.,jr.,NaN,NaN,NaN,NaN,NaN
768,Grear,J.W.,Jr.,NaN,NaN,NaN,NaN,NaN
21749,Grear,J.W.,Jr.,NaN,NaN,NaN,NaN,NaN



----------------------------------------
show names with **dropping_particle** found 0 records:



,family,given,suffix,particle,dropping_particle,nick,appellation,title



----------------------------------------
show names with **appellation** found 1 records:



,family,given,suffix,particle,dropping_particle,nick,appellation,title
13817,Sennen,NaN,NaN,NaN,NaN,NaN,Fr,NaN


Compile `canonical_string…` for the collector data we will later match the WikiData names with:

In [6]:
collectors['canonical_string_collector_parsed'] = (
  # use collectors.family only where given name has NA values, otherwise use family name + given name
  collectors.family.where(
      # condition
      collectors.given.isna(),
      # any other 
      # TODO improve the combined name for canonical_string_collector_parsed if any of the other dwc_parsed fields is not NaN
      other= (collectors.family + ", " + collectors.given) \
        if any(collectors.particle.isna()) \
        else collectors.particle + " " + collectors.family + ", " + collectors.given
  )
)
# move canonical_string_collector_parsed after column title (title was the last of the parsing columns)
col = collectors.pop("canonical_string_collector_parsed")
collectors.insert(collectors.columns.get_loc('title') + 1, col.name, col)

these_columns=["family", "given", "suffix", "particle", "dropping_particle", "nick", "appellation", "title", 'canonical_string_collector_parsed']

if 'source_data' in collectors.columns:
    these_columns.append("source_data")

display(collectors.tail().get(these_columns))


,family,given,suffix,particle,dropping_particle,nick,appellation,title,canonical_string_collector_parsed,source_data
54218,Ždanova,O.,NaN,NaN,NaN,NaN,NaN,NaN,"Ždanova, O.","Ždanova,O."
27655,Ždanova,O.,NaN,NaN,NaN,NaN,NaN,NaN,"Ždanova, O.","Lomonosova,M., Ždanova,O. & Šaulo,D."
54219,Žíla,V.,NaN,NaN,NaN,NaN,NaN,NaN,"Žíla, V.","Žíla,V."
54220,Волкова,Е.,NaN,NaN,NaN,NaN,NaN,NaN,"Волкова, Е.",Волкова Е.
54221,Жирова,O.,NaN,NaN,NaN,NaN,NaN,NaN,"Жирова, O.","Жирова,O."


In [7]:
# group and aggregate data to have unique name rows only for the matching of names later on
collectors_unique=collectors.groupby(['canonical_string_collector_parsed']).agg(
    family=('family', lambda x: list(x)[0]),
    given=('given', lambda x: list(x)[0]),
    suffix=('suffix', lambda x: list(x)[0]),
    particle=('particle', lambda x: list(x)[0]),
    dropping_particle=('dropping_particle', lambda x: list(x)[0]),
    nick=('nick', lambda x: list(x)[0]),
    appellation=('appellation', lambda x: list(x)[0]),
    title=('title', lambda x: list(x)[0]),
    source_data=('source_data', lambda x: list(x)[0]),
    occurrenceID_collectors_count= ('occurrenceID_count', 'sum'), # use count function
    occurrenceID_collectors_firstsample=('occurrenceID_firstsample', lambda x: list(x)[0]), # custom function, to get the first entry
    collectors_eventDate_mean=('eventDate_mean', 'mean'),
    collectors_eventDate_min=('eventDate_min', 'min'),
    collectors_eventDate_max=('eventDate_max', 'max')
).reset_index()

# move canonical_string_collector_parsed after column title
col = collectors_unique.pop("canonical_string_collector_parsed")
collectors_unique.insert(collectors_unique.columns.get_loc('title') + 1, col.name, col)

display(collectors_unique)

# column naming perhaps more clear (because we condensed the data)?
# collectors=collectors.add_suffix('_namegrouped') \
#  if not any(col.endswith("_namegrouped") for col in list(collectors.columns))

,family,given,suffix,particle,dropping_particle,nick,appellation,title,canonical_string_collector_parsed,source_data,occurrenceID_collectors_count,occurrenceID_collectors_firstsample,collectors_eventDate_mean,collectors_eventDate_min,collectors_eventDate_max
0,A. Cano,E.,NaN,NaN,NaN,NaN,NaN,NaN,"A. Cano, E.","A. Cano,E.",1,https://herbarium.bgbm.org/object/B100699397,2008-06-05 00:00:00,2008-06-05,2008-06-05
1,Aaiki,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Aaiki,Murashige & Aaiki,1,https://herbarium.bgbm.org/object/B101149305,1949-03-02 00:00:00,1949-03-02,1949-03-02
2,Aaronsohn,A.,NaN,NaN,NaN,NaN,NaN,NaN,"Aaronsohn, A.","Aaronsohn,A.",1,https://herbarium.bgbm.org/object/B100379341,1908-06-20 00:00:00,1908-06-20,1908-06-20
3,Abaouz,A.,NaN,NaN,NaN,NaN,NaN,NaN,"Abaouz, A.","Jury,S.L., Abaouz,A. ,Lafkih,M.Ait & Griffiths...",5,https://herbarium.bgbm.org/object/B100217620,1997-07-10 02:00:00,1997-07-05,1997-07-14
4,Abarca,R.,NaN,NaN,NaN,NaN,NaN,NaN,"Abarca, R.","Morales,J.F. & Abarca,R.",1,https://herbarium.bgbm.org/object/B101153811,2002-03-24 00:00:00,2002-03-24,2002-03-24
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18304,Żelazny,J.,NaN,NaN,NaN,NaN,NaN,NaN,"Żelazny, J.","Żelazny,J.",4,https://herbarium.bgbm.org/object/B100344466,2004-07-18 12:00:00,2004-06-09,2004-09-02
18305,Ždanova,O.,NaN,NaN,NaN,NaN,NaN,NaN,"Ždanova, O.","Ždanova,O.",6,https://herbarium.bgbm.org/object/B100263330,1989-01-24 19:12:00,1986-08-18,1989-09-10
18306,Žíla,V.,NaN,NaN,NaN,NaN,NaN,NaN,"Žíla, V.","Žíla,V.",3,https://herbarium.bgbm.org/object/B100009590,2003-05-17 08:00:00,2000-08-29,2004-09-24
18307,Волкова,Е.,NaN,NaN,NaN,NaN,NaN,NaN,"Волкова, Е.",Волкова Е.,1,https://herbarium.bgbm.org/object/B100530714,1951-07-18 00:00:00,1951-07-18,1951-07-18


## Set Up the Cosine Similarity and Text Search

See 
- for the application code https://github.com/nielsklazenga/avh-collectors/blob/master/cosine_similarity.ipynb
- for reading on the topic: Taylor, Josh. 2019. ‘Fuzzy Matching at Scale’. Towards Data Science (blog). 2 July 2019. https://towardsdatascience.com/fuzzy-matching-at-scale-84f2bfd0c536.

The `ngrams`-function is used as an analyzer in the text search later.

In [8]:
import pandas as pd, numpy as np, re
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import csr_matrix
from sparse_dot_topn import awesome_cossim_topn # pip install sparse-dot-topn

def get_matches_df(sparse_matrix, A, B, top=100):
    non_zeros = sparse_matrix.nonzero()

    sparserows = non_zeros[0]
    sparsecols = non_zeros[1]

    if top:
        nr_matches = top
    else:
        nr_matches = sparsecols.size

    left_side = np.empty([nr_matches], dtype=object)
    right_side = np.empty([nr_matches], dtype=object)
    similarity = np.zeros(nr_matches)

    for index in range(0, nr_matches):
        left_side[index] = A[sparserows[index]]
        right_side[index] = B[sparsecols[index]]
        similarity[index] = round(sparse_matrix.data[index], 3)

    return pd.DataFrame({'namematch_source_data': left_side,
                         'namematch_resource_data': right_side,
                         'namematch_similarity': similarity})

!pip install ftfy
from ftfy import fix_text

def ngrams(string, n=3):
    """
    Construct ngram(s) of a given text

    @param string: the text string to perform the ngram splitting on
    @param n: character length of the particular (split) result text each
    @return: string as ngram
    """
    string = fix_text(string) # fix text
    string = string.encode("ascii", errors="ignore").decode() #remove non ascii chars
    string = string.lower()
    chars_to_remove = [")","(","|","[","]","{","}","'"]
    rx = '[' + re.escape(''.join(chars_to_remove)) + ']'
    string = re.sub(rx, '', string)
    string = string.replace('&', 'and')
    string = string.replace(',', ' ')
    string = string.replace('-', ' ')
    string = string.replace('.', ' ')
    string = string.title()  # normalise case - capital at start of each word
    string = re.sub(' +', ' ', string).strip() # get rid of multiple spaces and replace with a single
    string = ' ' + string + ' '  # pad names for ngrams...
    string = re.sub(r'[,-./]|\sBD',r'', string)
    string = string.strip()
    this_ngrams = zip(*[string[i:] for i in range(n)])
    return [''.join(ngram) for ngram in this_ngrams]

error: externally-managed-environment

× This environment is externally managed
╰─> To install Python packages system-wide, try 'pacman -S
    python-xyz', where xyz is the package you are trying to
    install.
    
    If you wish to install a non-Arch-packaged Python package,
    create a virtual environment using 'python -m venv path/to/venv'.
    Then use path/to/venv/bin/python and path/to/venv/bin/pip.
    
    If you wish to install a non-Arch packaged Python application,
    it may be easiest to use 'pipx install xyz', which will manage a
    virtual environment for you. Make sure you have python-pipx
    installed via pacman.

note: If you believe this is a mistake, please contact your Python installation or OS distribution provider. You can override this, at the risk of breaking your Python installation or OS, by passing --break-system-packages.
hint: See PEP 668 for the detailed specification.


In [9]:
print("Show ngram examples:")
print("- simple name:", ngrams('Klazenga, N.'))
print("- data from collectors:", ngrams(collectors_unique["canonical_string_collector_parsed"].at[1])) 
print("- data from match-test:", ngrams(wd_matchtest['canonical_string'].at[0]))
print("- data from match-test (full name):", ngrams(wd_matchtest_fullnames['canonical_string_fullname'].at[3]))


Show ngram examples:
- simple name: ['Kla', 'laz', 'aze', 'zen', 'eng', 'nga', 'ga ', 'a N']
- data from collectors: ['Aai', 'aik', 'iki']
- data from match-test: ['Wal', 'alr', 'lra', 'rae', 'aev', 'eve', 'ven', 'ens', 'ns ', 's O', ' O ', 'O H']
- data from match-test (full name): ['188', '888', '881', '819', '197', '973', '73 ', '3 G', ' Ge', 'Geo', 'eor', 'org', 'rge', 'ges', 'es ', 's A', ' An', 'And', 'ndr']


In [10]:
# some example data
for i, row in enumerate(range(5)):
    if (i == 0):
        print('(WikiData’s) canonical_string = (constructed) canonical_string_fullname') 
    pprint.pprint("%s = %s" % (
        wd_matchtest['canonical_string'].at[row],
        wd_matchtest_fullnames['canonical_string_fullname'].at[row]
    ))


(WikiData’s) canonical_string = (constructed) canonical_string_fullname
'(-Walraevens), O.H. = (-Walraevens), O Heylen'
'(1835-1906), G.A.F.E. = (1835-1906), Gustav Adolf Ferdinand Eichler'
'(1873-1926), S.S. = (1873-1926), Søren Sørensen'
'(1888–1973), G.A. = (1888–1973), Georges André'
'(1904-1990), J.J. = (1904-1990), Johannes Johannessen'


In [11]:
def calculateTFIDFmatchingOfData(query_data, match_data, cossim_ntop=1, cossim_lower_bound=0.5):
    """
    Calculate a TF-IDF (Term Frequency — Inverse Document Frequency) matching with awesome_cossim_topn() and return matched data

    @param query_data: DataFrame usually a pandas data column to query names or strings for
    @param match_data: DataFrame against to match with
    @param cossim_ntop: how many cossim matches each shall be calculated (default 1, i.e. the highest similarity) — increase it to get more alternative
        matches with less similarity
    @param cossim_lower_bound: where is the lower similarity cut off to regard data as similar (default 0.5)

    @requires get_get_matches_df()
    @requires ngrams()
    @requires awesome_cossim_topn()
    @requires TfidfVectorizer()

    @return: a data frame dictionary: namematch_source_data, namematch_resource_data, namematch_similarity (from @see get_matches_df())
    @rtype pd.DataFrame
    """

    import time
    time_start = time.time()

    # Vectorize Wikidata name (use fit_transform())
    print('Vectorizing data. This may take a while...')
    vectorizer = TfidfVectorizer(min_df=1, analyzer=ngrams)
    tf_idf_matrix_clean = vectorizer.fit_transform(match_data)
    # Vectorize collectors’ names (use transform())
    tf_idf_matrix_dirty = vectorizer.transform(query_data)

    duration = time.time() - time_start
    print('Vectorizing completed: created a matrix of TF-IDF features after %s s' % duration)

    # Calculate Cosine Similarity; keep only the best match (ntop=1) and only if the similarity is greater than 0.5 (lower_bound=0.5)
    # (lower_bound: a threshold that the element of A*B must be greater than
    #  https://github.com/ing-bank/sparse_dot_topn/blob/3f40611b0553b50c27f23c7dcffc3ca9a9e8f5b5/sparse_dot_topn/awesome_cossim_topn.py#L26C9-L26C78)
    cossim_matches = awesome_cossim_topn(
        tf_idf_matrix_dirty,
        tf_idf_matrix_clean.transpose(),
        ntop=cossim_ntop,
        lower_bound=cossim_lower_bound
    )
    print("Cossim matches calculated after %s s" % (time.time() - time_start))

    print("Get all matches together ...")
    # construct the matching data frame
    matches_df = get_matches_df(
        cossim_matches,
        query_data,
        match_data,
        top=0
    )
    print("Done. Matches calculated after %s s" % (time.time() - time_start))

    return matches_df

In [12]:
criterion_fullnames = collectors_unique.given.str.contains('^\w{3,}', na=False)
collectors_names = collectors_unique['canonical_string_collector_parsed'][[not fullname for fullname in criterion_fullnames]].values

matches = calculateTFIDFmatchingOfData(
    collectors_names, 
    wd_matchtest['canonical_string'], 
    cossim_ntop=1 # e.g. cossim_ntop=3 would give more alternative matches as well, having lower similarities, data would increase 3 times as well
)
matches = matches.sort_values(by=['namematch_similarity'], ascending=[False])
matches = matches.reset_index(names=['old_index'])
display(matches)

Vectorizing data. This may take a while...
Vectorizing completed: created a matrix of TF-IDF features after 3.2568578720092773 s
Cossim matches calculated after 3.760153293609619 s
Get all matches together ...
Done. Matches calculated after 3.8888978958129883 s


,old_index,namematch_source_data,namematch_resource_data,namematch_similarity
0,15489,"Žíla, V.","Žíla, V.",1.0
1,4520,"Gerstner, J.","Gerstner, J.",1.0
2,4538,"Ghesquière, J.","Ghesquière, J.",1.0
3,4536,"Ghazanfar, S.A.","Ghazanfar, S.A.",1.0
4,10994,"Quintanar, A.","Quintanar, A.",1.0
...,...,...,...,...
15485,9952,Ohnesorge,"Desor, É.",0.5
15486,3683,"Fabbri, F.","Fabri, R.",0.5
15487,7628,"Latelo, M.G.","Melo, M.A.",0.5
15488,361,"Ani, H.","Anşin, R.",0.5


In [13]:
# criterion_fullnames = collectors_unique.given.str.contains('^\w{3,}', na=False)
collectors_fullnames = collectors_unique['canonical_string_collector_parsed'][criterion_fullnames].values
matches_fullnames = calculateTFIDFmatchingOfData(
    collectors_fullnames, 
    wd_matchtest_fullnames['canonical_string_fullname'], 
    cossim_ntop=1 # 10 would give more alternative matches also with lesser similarity
)

matches_fullnames = matches_fullnames.sort_values(by=['namematch_similarity'], ascending=[False])
matches_fullnames = matches_fullnames.reset_index(names=['old_index'])

display(matches_fullnames)

Vectorizing data. This may take a while...
Vectorizing completed: created a matrix of TF-IDF features after 3.435199499130249 s
Cossim matches calculated after 3.4989991188049316 s
Get all matches together ...
Done. Matches calculated after 3.5031869411468506 s


,old_index,namematch_source_data,namematch_resource_data,namematch_similarity
0,172,"Lagger, Franz Joseph","Lagger, Franz Joseph",1.000
1,197,"Mayer, Sebastian","Mayer, Sebastian",1.000
2,180,"Lindberg, Harald","Lindberg, Harald",1.000
3,181,"Lindfors, Axel Matias","Lindfors, Axel Matias",1.000
4,182,"Liu, Zheng-yu","Liu, Zheng Yu",1.000
...,...,...,...,...
340,289,"Sharpe, Guymer","Guymer, Gordon P",0.505
341,17,"Bayern, Therese","Bayer, L",0.503
342,225,"Nicomed, Rastern","Rastern, Nikomed",0.502
343,176,"Leonis, Christos","Kapellos, Christos",0.502


### Create Output Results

Combine the matches data frame back to the (BGBM) collectors and Wikidata items …

Note: merging 18.770.000 collector matches earlier to wikidata was too much to calculate. Hence the descision was to make the data unique by canonical_string_collector_parsed.

In [14]:
# # join (only) abbreviated name matches with collector source data
collectors_matches = pd.merge(
    collectors_unique, matches, 
    left_on='canonical_string_collector_parsed', right_on='namematch_source_data', 
    how='left'
)

collectors_matches.dropna(subset=['namematch_similarity'], inplace=True) # drop all matches that the method gave a NA (we narrowed the similarity range from 0.5 to 1.0)
collectors_matches # 17552 rows × 15 columns

,family,given,suffix,particle,dropping_particle,nick,appellation,title,canonical_string_collector_parsed,source_data,occurrenceID_collectors_count,occurrenceID_collectors_firstsample,collectors_eventDate_mean,collectors_eventDate_min,collectors_eventDate_max,old_index,namematch_source_data,namematch_resource_data,namematch_similarity
0,A. Cano,E.,NaN,NaN,NaN,NaN,NaN,NaN,"A. Cano, E.","A. Cano,E.",1,https://herbarium.bgbm.org/object/B100699397,2008-06-05 00:00:00,2008-06-05,2008-06-05,0.0,"A. Cano, E.","Cano, E.B.",0.664
1,Aaiki,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Aaiki,Murashige & Aaiki,1,https://herbarium.bgbm.org/object/B101149305,1949-03-02 00:00:00,1949-03-02,1949-03-02,1.0,Aaiki,"Naiki, A.",0.707
2,Aaronsohn,A.,NaN,NaN,NaN,NaN,NaN,NaN,"Aaronsohn, A.","Aaronsohn,A.",1,https://herbarium.bgbm.org/object/B100379341,1908-06-20 00:00:00,1908-06-20,1908-06-20,2.0,"Aaronsohn, A.","Aaronsohn, A.",1.000
4,Abarca,R.,NaN,NaN,NaN,NaN,NaN,NaN,"Abarca, R.","Morales,J.F. & Abarca,R.",1,https://herbarium.bgbm.org/object/B101153811,2002-03-24 00:00:00,2002-03-24,2002-03-24,3.0,"Abarca, R.","Abarca, L.",0.879
5,Abarca,R.J.,NaN,NaN,NaN,NaN,NaN,NaN,"Abarca, R.J.","Morales,J.F. & Abarca,R.J.",15,https://herbarium.bgbm.org/object/B101139201,2001-11-29 04:48:00,1998-12-20,2002-12-01,4.0,"Abarca, R.J.","Abarca, L.",0.800
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18301,Ţopa,E.,NaN,NaN,NaN,NaN,NaN,NaN,"Ţopa, E.","Ţopa,E. & Berlo,L.",4,https://herbarium.bgbm.org/object/B100124910,1972-10-23 12:00:00,1968-08-02,1978-12-29,15485.0,"Ţopa, E.","Țopa, E.",1.000
18302,Żarnowiec,J.,NaN,NaN,NaN,NaN,NaN,NaN,"Żarnowiec, J.","Jȩdrzejko,K., Żarnowiec,J. & Klama,H.",7,https://je.jacq.org/JE04006443,1985-06-02 20:00:00,1982-03-05,1987-10-10,15486.0,"Żarnowiec, J.","Żarnowiec, J.T.",0.943
18303,Żelany,J.,NaN,NaN,NaN,NaN,NaN,NaN,"Żelany, J.","Żelany,J.",1,https://herbarium.bgbm.org/object/B100220196,2004-06-28 00:00:00,2004-06-28,2004-06-28,15487.0,"Żelany, J.","Ważny, J.",0.670
18305,Ždanova,O.,NaN,NaN,NaN,NaN,NaN,NaN,"Ždanova, O.","Ždanova,O.",6,https://herbarium.bgbm.org/object/B100263330,1989-01-24 19:12:00,1986-08-18,1989-09-10,15488.0,"Ždanova, O.","Baranova, O.G.",0.599


In [15]:
# join (only) full name matches with collector source data
collectors_matches_fullname = pd.merge(
    collectors_unique, matches_fullnames, 
    left_on='canonical_string_collector_parsed' , right_on='namematch_source_data'
    #, suffixes=(None, '_namematch') # append to left-data, right-data only when identical column names occur
)

collectors_matches_fullname # 429 rows × 15 columns

,family,given,suffix,particle,dropping_particle,nick,appellation,title,canonical_string_collector_parsed,source_data,occurrenceID_collectors_count,occurrenceID_collectors_firstsample,collectors_eventDate_mean,collectors_eventDate_min,collectors_eventDate_max,old_index,namematch_source_data,namematch_resource_data,namematch_similarity
0,Abreu,Guilherme,NaN,de,NaN,NaN,NaN,NaN,"Abreu, Guilherme",Guilherme de Abreu (no. 103),1,http://id.snsb.info/snsb/collection/22086/3086...,1938-08-24 00:00:00,1938-08-24,1938-08-24,0,"Abreu, Guilherme","Rau, Guilherme",0.678
1,Adá,García,NaN,NaN,NaN,NaN,NaN,NaN,"Adá, García","García Adá, Luceño, Rico,E., Romero,T. & Varga...",1,https://herbarium.bgbm.org/object/B100296455,1987-06-19 00:00:00,1987-06-19,1987-06-19,1,"Adá, García","Adá, Ramón García",0.651
2,Allorge,Valia Selitsky,NaN,NaN,NaN,NaN,NaN,NaN,"Allorge, Valia Selitsky","Allorge,Valia (Valentine) Selitsky & Allorge,P.",1,https://je.jacq.org/JE04004935,1937-08-24 00:00:00,1937-08-24,1937-08-24,2,"Allorge, Valia Selitsky","Allorge, Valentine",0.514
3,Almquist,Emil,NaN,NaN,NaN,NaN,NaN,NaN,"Almquist, Emil","Almquist,Emil",1,https://dr.jacq.org/DR047889,1905-08-06 00:00:00,1905-08-06,1905-08-06,3,"Almquist, Emil","Almquist, Erik",0.793
4,Alschinger,Andreas,NaN,NaN,NaN,NaN,NaN,NaN,"Alschinger, Andreas","Alschinger,Andreas",1,https://herbarium.bgbm.org/object/B100749292,1847-01-01 00:00:00,1847-01-01,1847-01-01,4,"Alschinger, Andreas","Alschinger, Andreas",1.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
340,Zahn,Gustav,NaN,NaN,NaN,NaN,NaN,NaN,"Zahn, Gustav","Zahn,Gustav",1,https://je.jacq.org/JE00016825,1882-07-15 00:00:00,1882-07-15,1882-07-15,340,"Zahn, Gustav","Mann, Gustav",0.617
341,Zernig,Kurt,NaN,NaN,NaN,NaN,NaN,NaN,"Zernig, Kurt","Zernig,Kurt",3,https://herbarium.bgbm.org/object/B101139614,2019-06-29 08:00:00,2019-05-07,2019-09-28,341,"Zernig, Kurt","Zernig, Kurt",1.000
342,Zhuo,Zhou,NaN,NaN,NaN,NaN,NaN,NaN,"Zhuo, Zhou","Kilian,N., Raab-Straube,E. von, Zhang Jianwen,...",63,https://herbarium.bgbm.org/object/B100517202,2012-08-26 04:57:08,2012-08-17,2012-09-02,342,"Zhuo, Zhou","Zhou, Zhuo",0.725
343,Zickendrath,Ernst,NaN,NaN,NaN,NaN,NaN,NaN,"Zickendrath, Ernst","Zickendrath,Ernst",4,https://je.jacq.org/JE04006629,1898-10-14 16:00:00,1894-04-03,1900-09-01,343,"Zickendrath, Ernst","Zickendrath, Ernst",1.000


In [16]:
# join all name matches together
collectors_all_matches=pd.concat([collectors_matches, collectors_matches_fullname])
collectors_all_matches.sort_values(by=['namematch_similarity', 'family'], ascending=[False, True], inplace=True)
collectors_all_matches.head()

,family,given,suffix,particle,dropping_particle,nick,appellation,title,canonical_string_collector_parsed,source_data,occurrenceID_collectors_count,occurrenceID_collectors_firstsample,collectors_eventDate_mean,collectors_eventDate_min,collectors_eventDate_max,old_index,namematch_source_data,namematch_resource_data,namematch_similarity
2,Aaronsohn,A.,NaN,NaN,NaN,NaN,NaN,NaN,"Aaronsohn, A.","Aaronsohn,A.",1,https://herbarium.bgbm.org/object/B100379341,1908-06-20 00:00:00,1908-06-20,1908-06-20,2.0,"Aaronsohn, A.","Aaronsohn, A.",1.0
7,Abbe,E.C.,NaN,NaN,NaN,NaN,NaN,NaN,"Abbe, E.C.","Raup,H.M. & Abbe,E.C.",2,https://herbarium.bgbm.org/object/B100241637,1946-05-22 12:00:00,1932-07-26,1960-03-18,6.0,"Abbe, E.C.","Abbe, E.C.",1.0
10,Abbott,J.R.,NaN,NaN,NaN,NaN,NaN,NaN,"Abbott, J.R.","Bécquer,E. & Abbott,J.R.",80,https://herbarium.bgbm.org/object/B100181131,1999-11-05 15:33:58,1992-05-20,2007-07-17,8.0,"Abbott, J.R.","Abbott, J.R.",1.0
12,Abbott,W.L.,NaN,NaN,NaN,NaN,NaN,NaN,"Abbott, W.L.","Abbott, W.L. (no. 1248)",4,http://id.snsb.info/snsb/collection/504820/626...,1921-08-04 16:00:00,1921-04-05,1922-04-27,10.0,"Abbott, W.L.","Abbott, W.L.",1.0
17,Abedin,S.,NaN,NaN,NaN,NaN,NaN,NaN,"Abedin, S.","Abedin,S. & Hussein,A.",13,https://herbarium.bgbm.org/object/B100046632,1971-09-20 06:00:00,1969-05-29,1972-05-26,13.0,"Abedin, S.","Abedin, S.",1.0


In [17]:
# Save the plain name matching results only ...
## cell split - code
# if not os.path.exists('data'):
#     print("Make data directory for saving …")
#     os.makedirs('data')
# 
# # Set some global varialbes
# # this_timestamp_for_data=time.strftime('%Y%m%d') # 20230719
# this_timestamp_for_data=20230821
# 
# this_output_file='data/results_bgbm_collectors_vs_wikidata-botanists_cossim-similarity_plain-names_%s.csv' % (
#     this_timestamp_for_data
# )
# 
# collectors_all_matches.to_csv(this_output_file)
# 
# print("Wrote plain name matches of collector names into %s (%d kB)" % 
#     (this_output_file, os.path.getsize(this_output_file) >> 10 ) # 10000 >> 10 = bitshift operator, to get kilo bytes (10-bits=>1024)
# )
## cell split - code
# old code # Join Wikidata items
# df_avh_matches_wikidata = pd.merge(df_avh_matches, df_wikidata                , left_on='namematch_resource_data', right_on='canonical_string', how='left')
# df_avh_matches_wikidata = pd.merge(df_avh_matches_wikidata, df_wikidata_unique, left_on='namematch_resource_data', right_on='canonical_string', how='left')
# df_avh_matches_wikidata.rename(columns={df_avh_matches_wikidata.columns.tolist()[-1]: 'dup_count'}, inplace=True)


In [18]:
# merge now with WikiData: the matching data and the wiki data’s on the conaonical string name
collectors_matches_tmp_names_abbreviated = pd.merge(
    collectors_matches, wikidata, 
    left_on='namematch_resource_data', right_on='canonical_string'
)
collectors_matches_tmp_fullnames = pd.merge(
    collectors_matches_fullname, wikidata, 
    left_on='namematch_resource_data', right_on='canonical_string_fullname'
)
collectors_matches_g1_merged_wikidata = pd.concat(
    [collectors_matches_tmp_names_abbreviated, collectors_matches_tmp_fullnames]
    , ignore_index=True
)

In [19]:
print("Show example data of «Kotschy…» with Cosine Similiarity we had 0.6 … 1.0 (Nearest Neighbour distances were from 0.0 to almost 1.0)")
print("There was a match of Kotschyi, C.G.T. → Kotschy, T.   → 0.614 → http://www.wikidata.org/wiki/Q113299  with lower similarity and a correct match to Carl Georg Theodor … :-)")
print("There was a match of Kotschy, C.G.T.  → Kotschy, C.F. → 0.824 → http://www.wikidata.org/entity/Q86842 with higer similarity but it is probably a wrong match of Carl Georg Theodor → to Carl Friedrich … :-/")

criterion = collectors_matches_g1_merged_wikidata['canonical_string_collector_parsed'].map(lambda x: x.startswith('Kotschy'))
collectors_matches_g1_merged_wikidata[criterion].get([
    # 'canonical_string_collector_parsed', # canonical_string_collector_parsed = namematch_source_data
    'occurrenceID_collectors_count', 'occurrenceID_collectors_firstsample',
    'namematch_source_data', 'namematch_resource_data', 'namematch_similarity', 
    # 'canonical_string_fullname', 
    'itemLabel', 'wikidata_link'
])

Show example data of «Kotschy…» with Cosine Similiarity we had 0.6 … 1.0 (Nearest Neighbour distances were from 0.0 to almost 1.0)
There was a match of Kotschyi, C.G.T. → Kotschy, T.   → 0.614 → http://www.wikidata.org/wiki/Q113299  with lower similarity and a correct match to Carl Georg Theodor … :-)
There was a match of Kotschy, C.G.T.  → Kotschy, C.F. → 0.824 → http://www.wikidata.org/entity/Q86842 with higer similarity but it is probably a wrong match of Carl Georg Theodor → to Carl Friedrich … :-/


,occurrenceID_collectors_count,occurrenceID_collectors_firstsample,namematch_source_data,namematch_resource_data,namematch_similarity,itemLabel,wikidata_link
8041,1,http://id.snsb.info/snsb/collection/22980/3175...,"Kotschy, C.G.","Kotschy, C.F.",0.895,Carl Friedrich Kotschy,http://www.wikidata.org/wiki/Q86842
8042,2215,http://id.snsb.info/snsb/collection/108230/167...,"Kotschy, C.G.T.","Kotschy, C.F.",0.824,Carl Friedrich Kotschy,http://www.wikidata.org/wiki/Q86842
8043,36,http://id.snsb.info/snsb/collection/16719/2549...,"Kotschy, K.G.T.","Kotschy, T.",0.723,Theodor Kotschy,http://www.wikidata.org/wiki/Q113299
8044,294,http://id.snsb.info/snsb/collection/117808/176...,"Kotschy, T.","Kotschy, T.",1.000,Theodor Kotschy,http://www.wikidata.org/wiki/Q113299
8045,3,https://herbarium.bgbm.org/object/B100526350,"Kotschy, Th","Kotschy, T.",0.888,Theodor Kotschy,http://www.wikidata.org/wiki/Q113299
8046,1,https://herbarium.bgbm.org/object/B100160086,"Kotschyi, C.G.T.","Kotschy, T.",0.614,Theodor Kotschy,http://www.wikidata.org/wiki/Q113299
16850,1,https://herbarium.bgbm.org/object/B101086119,"Kotschy, Carl Georg Theodor","Kotschy, Theodor",0.746,Theodor Kotschy,http://www.wikidata.org/wiki/Q113299


In [20]:
pprint.pprint(collectors_matches_g1_merged_wikidata.columns)

Index(['family', 'given', 'suffix', 'particle', 'dropping_particle', 'nick',
       'appellation', 'title', 'canonical_string_collector_parsed',
       'source_data', 'occurrenceID_collectors_count',
       'occurrenceID_collectors_firstsample', 'collectors_eventDate_mean',
       'collectors_eventDate_min', 'collectors_eventDate_max', 'old_index',
       'namematch_source_data', 'namematch_resource_data',
       'namematch_similarity', 'item', 'itemLabel', 'surname', 'initials',
       'canonical_string', 'canonical_string_fullname', 'orcid', 'viaf',
       'isni', 'harv', 'ipni', 'abbr', 'bionomia_id', 'yob', 'yod', 'wyb',
       'wye', 'wikidata_link', 'orcid_link', 'harv_link', 'ipni_link',
       'bionomia_link'],
      dtype='object')


In [21]:
collectors_matches_g1_merged_wikidata.head()

,family,given,suffix,particle,dropping_particle,nick,appellation,title,canonical_string_collector_parsed,source_data,...,bionomia_id,yob,yod,wyb,wye,wikidata_link,orcid_link,harv_link,ipni_link,bionomia_link
0,A. Cano,E.,NaN,NaN,NaN,NaN,NaN,NaN,"A. Cano, E.","A. Cano,E.",...,Q42335752,1964,2021,<NA>,<NA>,http://www.wikidata.org/wiki/Q42335752,https://orcid.org/0000-0003-3529-9439,NaN,NaN,https://bionomia.net/Q42335752
1,Cano-E,A.A.,NaN,NaN,NaN,NaN,NaN,NaN,"Cano-E, A.A.","Weigend,M., Hilger,H.H. & Cano-E.,A.A.",...,Q42335752,1964,2021,<NA>,<NA>,http://www.wikidata.org/wiki/Q42335752,https://orcid.org/0000-0003-3529-9439,NaN,NaN,https://bionomia.net/Q42335752
2,Aaiki,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Aaiki,Murashige & Aaiki,...,NaN,<NA>,<NA>,<NA>,<NA>,http://www.wikidata.org/wiki/Q33686006,NaN,NaN,https://www.ipni.org/a/20029813-1,NaN
3,Aaronsohn,A.,NaN,NaN,NaN,NaN,NaN,NaN,"Aaronsohn, A.","Aaronsohn,A.",...,Q2086130,1876,1919,<NA>,<NA>,http://www.wikidata.org/wiki/Q2086130,NaN,https://kiki.huh.harvard.edu/databases/botanis...,https://www.ipni.org/a/23-1,https://bionomia.net/Q2086130
4,Abarca,R.,NaN,NaN,NaN,NaN,NaN,NaN,"Abarca, R.","Morales,J.F. & Abarca,R.",...,NaN,<NA>,<NA>,<NA>,<NA>,http://www.wikidata.org/wiki/Q36610614,NaN,https://kiki.huh.harvard.edu/databases/botanis...,https://www.ipni.org/a/34769-1,NaN


In [22]:
# # Select useful columns for data results
# collectors_wikidata_cossim = collectors_matches_g1_merged_wikidata[
#     ['canonical_string_collector_parsed', 'family', 'given', 
#      'occurrenceID_collectors_count', 'occurrenceID_collectors_firstsample',
#      'source_data',
#     'namematch_source_data', 'namematch_resource_data', 'namematch_similarity', 
#     'item', 'canonical_string', 'itemLabel',
#     'orcid', 'viaf', 'isni', 'harv', 'ipni', 'abbr', 'bionomia_id', 'yob', 'yod', 'wyb']
# ]
# 
# # Order by similarity (desc), number of Wikidata items (asc) and number of collections (desc)
# collectors_wikidata_cossim.sort_values(by=['namematch_similarity', 'family'], ascending=[False, True], inplace=True)
# 
# collectors_wikidata_cossim
## cell split - code
# Kotschy example again with all merged columns
# pd.set_option("display.max_columns", None) # default ?20
#
# criterion = collectors_wikidata_cossim['canonical_string_collector_parsed'].map(lambda x: x.startswith('Kotschy'))
# print("Show example of «Kotschy…» with similarities of 0.5 … 1.0")
# print("There was a match of Kotschy, C.G.T. → Kotschy, C.F. → 0.824 → http://www.wikidata.org/entity/Q86842 and it is probably a wrong match of Carl Georg Theodor → to Carl Friedrich … :-/")
# collectors_wikidata_cossim[criterion]
## cell split - code

# TODO further evaluation or filtering, counting, clean up aso.
# if not os.path.exists('data'):
#     os.makedirs('data')
# 
# # bgbm_collectors_cosine-similarity_wikidata-botanists_%s.csv
# this_output_file='data/results_bgbm_collectors_vs_wikidata-botanists_cossim-similarity_merged-data_%s.csv' % (
#     this_timestamp_for_data
# )
# 
# collectors_wikidata_cossim.to_csv(this_output_file)
# 
# print("Wrote matches of collector names into %s (%d kB)" % 
#     (this_output_file, os.path.getsize(this_output_file) >> 10 ) # 10000 >> 10 = bitshift operator, to get kilo bytes (10-bits=>1024)
# )

## Output Mapping to DarwinCore Attribution Output

Here we map table data fields to fields of DarwinCore Attribution (<https://github.com/tdwg/attribution/>, <https://github.com/tdwg/attribution/blob/master/people/dwc/agent_actions_v2020-09-08.xml>) 

## Scoring

Individual scored properties should actually be balanced in such a way that one can simply add up these different property scores; in this case, assessment of the calculated values is still necessary. The problem here with calculation with a distance measure is that we have the opposite of similarity, whose distance can become greater than 1, which must somehow be mapped to a scope of 0 … 1 (or -1 … 0 … 1) (TODO review).

General thoughts: With a score of -1 to 1, it can be assumed that:
* -1 means full devaluation or no agreement
* 1 means full upvoting or agreement, and
* 0 can have several interpretations: it is in between, or no rating possible, or missing values.

### Task to Be Solved in Evaluating the Life Time ~ Rating/Scoring

We have grouped the collection date (evenDate) to the name in the source data, so it may be that for (abbreviated) names, e.g. “Bachmann, F.”, the collection date is valid for *several* personal names, not just one. This must be taken into account when considering and evaluating whether the life data match the collection date. The rating of the life data has the following idea:

| Score (life time) | Remarks | 
|--|--|
| 1.0  | complete match                     |
| 0.5  | somewhat correct, but has errors or mistakes, indicating multiple person names    |
| 0.0   | no evaluation (or not possible) |
| -0.5 | is rather to be rejected, indicating multiple person names and possibly overlapping time spans of the collection date of different person names, or mistakes in the original data |
| -1.0 | completely rejected                |

### Task to Be Solved With Several Names ~ Assessment/Score

Since we do not know if there are other possible names somewhere when there is only one name, we cannot assign a “1” (= full agreement) with certainty, so it was decided that if only 1 name was found, this would be evaluated as zero, in the sense of no evaluation. So when evaluating the multiple names, only the mismatches are evaluated, according to the idea:

| Score (multiple names) | Remarks | 
|--|--|
| 1.0  | this value (=full upvoting or agreement) would never be set in this regard, since we do not know all the full names of the cosmos ;-), and could state this score certainty of 1.0 |
| 0.0 | no evaluation, because only 1 name found | 
| less than 0 | multiple names found, i.e. deduction (perhaps just -0.5, as a decision needs to be made) | 

---

TODO review interpretation:

- the fields are defined in <https://github.com/tdwg/attribution/blob/master/people/dwc/agent_actions_v2020-09-08.xml> and regarding from this DwC-attribution concept: is it correct to map it like the following (`name` would represent the *interpreted* resource name (in long format), not the *source* collector `name` (in (theoretically) long format))?
    ```
    name          ← itemLabel (wikiData)
    alternateName ← canonical_string_collector_parsed (actual collector name)
    ```

In [23]:
pprint.pprint(collectors_matches_g1_merged_wikidata.columns)

Index(['family', 'given', 'suffix', 'particle', 'dropping_particle', 'nick',
       'appellation', 'title', 'canonical_string_collector_parsed',
       'source_data', 'occurrenceID_collectors_count',
       'occurrenceID_collectors_firstsample', 'collectors_eventDate_mean',
       'collectors_eventDate_min', 'collectors_eventDate_max', 'old_index',
       'namematch_source_data', 'namematch_resource_data',
       'namematch_similarity', 'item', 'itemLabel', 'surname', 'initials',
       'canonical_string', 'canonical_string_fullname', 'orcid', 'viaf',
       'isni', 'harv', 'ipni', 'abbr', 'bionomia_id', 'yob', 'yod', 'wyb',
       'wye', 'wikidata_link', 'orcid_link', 'harv_link', 'ipni_link',
       'bionomia_link'],
      dtype='object')


In [24]:
# TODO review score (similarity + ((yob,yod) ~ (eventDate_min, eventDate_max))

collectors_wikidata_cossim = collectors_matches_g1_merged_wikidata[
    ['canonical_string_collector_parsed', 'family', 'given', 
     'occurrenceID_collectors_count', 'occurrenceID_collectors_firstsample',
     'source_data',
    'namematch_source_data', 'namematch_resource_data', 'namematch_similarity', 
    'item', 'canonical_string', 'itemLabel',
    'orcid', 'viaf', 'isni', 'harv', 'ipni', 'abbr', 'bionomia_id', 
    'collectors_eventDate_mean', 'collectors_eventDate_min', 'collectors_eventDate_max',
     'yob', 'yod', 'wyb'
    ]
]

# Order by similarity (desc), number of Wikidata items (asc) and number of collections (desc)
collectors_wikidata_cossim.sort_values(
    by=['namematch_similarity', 'family', 'given'], 
    ascending=[False, True, True], 
    inplace=True
)

collectors_dwcagent_output=collectors_wikidata_cossim.get([
    "occurrenceID_collectors_firstsample", 
    "canonical_string_collector_parsed",
    'family', 'given',
    "namematch_similarity", 
    "source_data", 
    "itemLabel", 
    "item",
    "collectors_eventDate_min",
    "collectors_eventDate_max",
    'yob', 'yod'
]).copy()

collectors_dwcagent_output['canonical_string_collector_parsed'].replace(to_replace=r'([^,]+),\s*(.+)', value='\\2 \\1', inplace=True, regex=True)
collectors_dwcagent_output['namematch_similarity_annotation'] = collectors_dwcagent_output['namematch_similarity'].astype(str).str.replace(r'(.+)', '\\1 (cos similarity)', regex=True)
# collectors_dwcagent_output.insert(len(collectors_dwcagent_output.columns), 'namematch_similarity_annotation', '', allow_duplicates=True)

collectors_dwcagent_output.insert(len(collectors_dwcagent_output.columns), 'life_time_periode', '', allow_duplicates=True)

combine_life_times = lambda this_df: ("%s-%s" % (this_df["yob"], this_df["yod"])).replace(r"<NA>", "?")
collectors_dwcagent_output["life_time_periode"]=collectors_dwcagent_output.apply(combine_life_times, axis="columns")

# collectors_dwcagent_output["life_time_periode"]

years_from_birth_until_first_collection_activity = 10
collectors_dwcagent_output["yob_is_lt_eventDate_min"] = collectors_dwcagent_output["yob"] + years_from_birth_until_first_collection_activity < collectors_dwcagent_output["collectors_eventDate_min"].dt.year
collectors_dwcagent_output["yod_is_gt_eventDate_max"] = collectors_dwcagent_output["yod"] > collectors_dwcagent_output["collectors_eventDate_max"].dt.year
collectors_dwcagent_output["custom_score_lifetime"] = 0
collectors_dwcagent_output.insert(len(collectors_dwcagent_output.columns), 'custom_score_lifetime_annotation', '', allow_duplicates=True)

# df.loc[(df['column_of_interest'] … condition), 'fill_to_column'] = value 

collectors_dwcagent_output.loc[
    collectors_dwcagent_output["yob_is_lt_eventDate_min"] & collectors_dwcagent_output["yod_is_gt_eventDate_max"],
    "custom_score_lifetime"
] = 1
# True cases but <NA> missing values
collectors_dwcagent_output.loc[
    collectors_dwcagent_output["yob_is_lt_eventDate_min"] & collectors_dwcagent_output["yod_is_gt_eventDate_max"].isnull(),
    "custom_score_lifetime"
] = 1
collectors_dwcagent_output.loc[
    collectors_dwcagent_output["yob_is_lt_eventDate_min"].isnull() & collectors_dwcagent_output["yod_is_gt_eventDate_max"],
    "custom_score_lifetime"
] = 1
collectors_dwcagent_output.loc[
    collectors_dwcagent_output["yob_is_lt_eventDate_min"].isnull() & collectors_dwcagent_output["yod_is_gt_eventDate_max"].isnull(),
    "custom_score_lifetime"
] = 0

# False cases
collectors_dwcagent_output.loc[
    (collectors_dwcagent_output["yob_is_lt_eventDate_min"]==False) & (collectors_dwcagent_output["yod_is_gt_eventDate_max"] == False),
    "custom_score_lifetime"
] = -1
collectors_dwcagent_output.loc[
    (collectors_dwcagent_output["yob_is_lt_eventDate_min"]==True) & (collectors_dwcagent_output["yod_is_gt_eventDate_max"] == False),
    "custom_score_lifetime"
] = 0.5
collectors_dwcagent_output.loc[
    (collectors_dwcagent_output["yob_is_lt_eventDate_min"]==False) & (collectors_dwcagent_output["yod_is_gt_eventDate_max"] == True),
    "custom_score_lifetime"
] = 0.5

# False cases but <NA> missing values
collectors_dwcagent_output.loc[
    (collectors_dwcagent_output["yob_is_lt_eventDate_min"]==False) & (collectors_dwcagent_output["yod_is_gt_eventDate_max"].isnull()),
    "custom_score_lifetime"
] = -0.5
collectors_dwcagent_output.loc[
    (collectors_dwcagent_output["yob_is_lt_eventDate_min"].isnull()) & (collectors_dwcagent_output["yod_is_gt_eventDate_max"] == False),
    "custom_score_lifetime"
] = -0.5

# annotations True cases
collectors_dwcagent_output.loc[
    collectors_dwcagent_output["yob_is_lt_eventDate_min"] & collectors_dwcagent_output["yod_is_gt_eventDate_max"], 
    "custom_score_lifetime_annotation"
] = "full match"

# annotations True cases but <NA> missing values
collectors_dwcagent_output.loc[
    collectors_dwcagent_output["yob_is_lt_eventDate_min"] & collectors_dwcagent_output["yod_is_gt_eventDate_max"].isnull(), 
    "custom_score_lifetime_annotation"
] = "OK? year of death is missing"
collectors_dwcagent_output.loc[
    collectors_dwcagent_output["yob_is_lt_eventDate_min"].isnull() & collectors_dwcagent_output["yod_is_gt_eventDate_max"], 
    "custom_score_lifetime_annotation"
] = "OK? year of birth is missing"
collectors_dwcagent_output.loc[
    collectors_dwcagent_output["yob_is_lt_eventDate_min"].isnull() & collectors_dwcagent_output["yod_is_gt_eventDate_max"].isnull(), 
    "custom_score_lifetime_annotation"
] = "unknown life time"

# annotations False cases
collectors_dwcagent_output.loc[
    (collectors_dwcagent_output["yob_is_lt_eventDate_min"]==False) & (collectors_dwcagent_output["yod_is_gt_eventDate_max"] == False), 
    "custom_score_lifetime_annotation"
] = "life time not matching any enventDate (yob + %s … yod)" % years_from_birth_until_first_collection_activity
collectors_dwcagent_output.loc[
    (collectors_dwcagent_output["yob_is_lt_eventDate_min"]==True) & (collectors_dwcagent_output["yod_is_gt_eventDate_max"] == False), 
    "custom_score_lifetime_annotation"
] = "OK yob + %s, but yod not matching, check name and liftime data" % years_from_birth_until_first_collection_activity
collectors_dwcagent_output.loc[
    (collectors_dwcagent_output["yob_is_lt_eventDate_min"]==False) & (collectors_dwcagent_output["yod_is_gt_eventDate_max"] == True), 
    "custom_score_lifetime_annotation"
] = "yob + %s not matching, OK yod, check name and liftime data" % years_from_birth_until_first_collection_activity
# annotations False cases but <NA> missing values
collectors_dwcagent_output.loc[
    (collectors_dwcagent_output["yob_is_lt_eventDate_min"]==False) & (collectors_dwcagent_output["yod_is_gt_eventDate_max"].isnull()), 
    "custom_score_lifetime_annotation"
] = "yob + %s not matching, yod unknown, check name and liftime data" % years_from_birth_until_first_collection_activity
collectors_dwcagent_output.loc[
    (collectors_dwcagent_output["yob_is_lt_eventDate_min"].isnull()) & (collectors_dwcagent_output["yod_is_gt_eventDate_max"]==False), 
    "custom_score_lifetime_annotation"
] = "yob unknown, yod not matching, check name and liftime data"

collectors_dwcagent_output["custom_score_multiple_names"]=0 # 0 shall mean: we don’t know yet for real
collectors_dwcagent_output.loc[
    (collectors_dwcagent_output['canonical_string_collector_parsed'].duplicated(keep=False)),
    'custom_score_multiple_names'
] = -0.5 # one decision has to be made, so cut the range of -1 to 0 only into half (or include multiple count somehow?)

collectors_dwcagent_output['custom_score_overall'] = (
    collectors_dwcagent_output['namematch_similarity'] * \
    (
        ( collectors_dwcagent_output["custom_score_lifetime"] + collectors_dwcagent_output['custom_score_multiple_names']) / 2
    )
).round(3)

collectors_dwcagent_output['attributionRemarks'] = collectors_dwcagent_output.apply(
    lambda row: "{similarity_distance_note};"
                " {score_overall:.2f} (score overall);"
                " {lifetime_periode} (life time);"
                " {lifetime_score:.1f} (life time score);"
                " {lifetime_score_annote} (life time score note);"
                " {score_multinames:.2f} (score multiple names);"
        .format(
    similarity_distance_note=row['namematch_similarity_annotation'],
    lifetime_periode=row["life_time_periode"],
    lifetime_score=row["custom_score_lifetime"],
    lifetime_score_annote=row["custom_score_lifetime_annotation"],
    score_overall=row["custom_score_overall"],
    score_multinames=row["custom_score_multiple_names"]
    ), axis='columns'
)

# adjust dwcagent displayOrder also to olerall score
collectors_dwcagent_output.sort_values(
    by=['namematch_similarity', 'family', 'given', 'custom_score_overall'], 
    ascending=[False, True, True, False], 
    inplace=True
)
# use ordered canonical_string_collector_parsed to generate displayOrder
temp_duplicated = collectors_dwcagent_output['canonical_string_collector_parsed'].duplicated() 
    # duplicated() keeps the first value False and mark all other duplicats as True, i.e. we can cumulate the Trues, it gives the order index
temp_insert_value=temp_duplicated.groupby(collectors_dwcagent_output['canonical_string_collector_parsed']).cumsum() + 1 # display order starts at 1, incrementing
collectors_dwcagent_output.insert(collectors_dwcagent_output.columns.get_loc('canonical_string_collector_parsed') + 1, 'displayOrder', temp_insert_value, allow_duplicates=True)

# test an show example data
show_display_output=True
if show_display_output:
    display(collectors_dwcagent_output.loc[collectors_dwcagent_output['yob_is_lt_eventDate_min'] == True].get([
        # "occurrenceID_collectors_firstsample",
        "canonical_string_collector_parsed",
        'itemLabel',
        "custom_score_overall",
        "attributionRemarks",
        'custom_score_multiple_names',
        "namematch_similarity",
        # 'yob', 'yod',
        "life_time_periode", 
        'collectors_eventDate_min', 'collectors_eventDate_max', 
        "yob_is_lt_eventDate_min" ,'yod_is_gt_eventDate_max', 
        'custom_score_lifetime', 'custom_score_lifetime_annotation'
    ]).head(5))
    display(collectors_dwcagent_output.loc[collectors_dwcagent_output['yob_is_lt_eventDate_min'] == False].get([
        # "occurrenceID_collectors_firstsample",
        "canonical_string_collector_parsed",
        'itemLabel',
        "custom_score_overall",
        "attributionRemarks",
        'custom_score_multiple_names',
        "namematch_similarity",
        # 'yob', 'yod',
        "life_time_periode", 
        'collectors_eventDate_min', 'collectors_eventDate_max', 
        "yob_is_lt_eventDate_min" ,'yod_is_gt_eventDate_max', 
        'custom_score_lifetime', 'custom_score_lifetime_annotation'
    ]).head(5))

/tmp/ipykernel_24049/2497154979.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  collectors_wikidata_cossim.sort_values(


,canonical_string_collector_parsed,itemLabel,custom_score_overall,attributionRemarks,custom_score_multiple_names,namematch_similarity,life_time_periode,collectors_eventDate_min,collectors_eventDate_max,yob_is_lt_eventDate_min,yod_is_gt_eventDate_max,custom_score_lifetime,custom_score_lifetime_annotation
3,A. Aaronsohn,Aaron Aaronsohn,0.5,1.0 (cos similarity); 0.50 (score overall); 18...,0.0,1.0,1876-1919,1908-06-20,1908-06-20,True,True,1.0,full match
8,E.C. Abbe,Ernst Cleveland Abbe,0.5,1.0 (cos similarity); 0.50 (score overall); 19...,0.0,1.0,1905-2000,1932-07-26,1960-03-18,True,True,1.0,full match
10,J.R. Abbott,J. Richard Abbott,0.5,1.0 (cos similarity); 0.50 (score overall); 19...,0.0,1.0,1968-?,1992-05-20,2007-07-17,True,<NA>,1.0,OK? year of death is missing
12,W.L. Abbott,William Louis Abbott,0.5,1.0 (cos similarity); 0.50 (score overall); 18...,0.0,1.0,1860-1936,1921-04-05,1922-04-27,True,True,1.0,full match
16,S. Abedin,Sultanul Abedin,0.5,1.0 (cos similarity); 0.50 (score overall); 19...,0.0,1.0,1952-?,1969-05-29,1972-05-26,True,<NA>,1.0,OK? year of death is missing


,canonical_string_collector_parsed,itemLabel,custom_score_overall,attributionRemarks,custom_score_multiple_names,namematch_similarity,life_time_periode,collectors_eventDate_min,collectors_eventDate_max,yob_is_lt_eventDate_min,yod_is_gt_eventDate_max,custom_score_lifetime,custom_score_lifetime_annotation
96,P. Aellen,Paul Aellen,0.25,1.0 (cos similarity); 0.25 (score overall); 18...,0.0,1.0,1896-1973,1848-04-28,1969-10-11,False,True,0.5,"yob + 10 not matching, OK yod, check name and ..."
142,S. Ahmad,Shamsuddin Ahmad,-0.50,1.0 (cos similarity); -0.50 (score overall); 1...,-0.5,1.0,1942-?,1934-09-01,1957-01-15,False,<NA>,-0.5,"yob + 10 not matching, yod unknown, check name..."
350,O. Anders,Olaf Anders,-0.25,1.0 (cos similarity); -0.25 (score overall); 1...,0.0,1.0,1931-?,1883-06-01,1973-08-30,False,<NA>,-0.5,"yob + 10 not matching, yod unknown, check name..."
376,T. Anderson,Trante-Heidi Anderson,-0.50,1.0 (cos similarity); -0.50 (score overall); 1...,-0.5,1.0,1946-?,1862-05-01,1862-08-26,False,<NA>,-0.5,"yob + 10 not matching, yod unknown, check name..."
397,Andrä,Édouard André,0.25,1.0 (cos similarity); 0.25 (score overall); 18...,0.0,1.0,1840-1911,1841-01-01,1895-10-01,False,True,0.5,"yob + 10 not matching, OK yod, check name and ..."


In [25]:
column_map_dwcagent = {
    'occurrenceID_collectors_firstsample':'occurrenceID',
    'canonical_string_collector_parsed':  'alternateName',
    'source_data':                        'verbatimName',
    'itemLabel':                          'name',
    'item':                               'identifier',
    'collectors_eventDate_min':           'startedAtTime',
    'collectors_eventDate_max':           'endedAtTime',
    'namematch_similarity':               'custom_namematch_similarity'
}
collectors_dwcagent_output.rename(
    mapper=column_map_dwcagent,
    axis='columns',
    inplace=True)

collectors_dwcagent_output.insert(collectors_dwcagent_output.columns.get_loc('identifier')          + 1, 'agentIdentifierType', 'wikidata' , allow_duplicates=True)
collectors_dwcagent_output.insert(collectors_dwcagent_output.columns.get_loc('agentIdentifierType') + 1, 'agentType'          , 'Person'   , allow_duplicates=True)
collectors_dwcagent_output.insert(collectors_dwcagent_output.columns.get_loc('identifier')          + 1, 'action'             , 'collected', allow_duplicates=True)

show_display_output=False
if show_display_output:
    collectors_dwcagent_output.head(20)

collectors_dwcagent_output=collectors_dwcagent_output.reindex(
    columns=[
        'occurrenceID', # no DwC agent standard (yet)?
        'verbatimName',
        'alternateName',
        'displayOrder', # shall start from 1, 2, 3 …
        'name',
        'attributionRemarks',
        'startedAtTime',
        'endedAtTime',
        'agentType',
        'action',
        'agentIdentifierType',
        'identifier',
        "custom_score_overall", # keep it for calculation convenience, no standard in DwC agent
        'custom_namematch_similarity',# keep it for calculation convenience, no standard in DwC agent
        'custom_score_multiple_names',# keep it for calculation convenience, no standard in DwC agent
        'custom_score_lifetime' # keep it for calculation convenience, no standard in DwC agent
    ]
)
# column deletion not neccessary after ….reindex(columns=[…])
# for this_column in ['yob', 'yod', 'life_time_periode', 'yob_is_lt_eventDate_min', 'yod_is_gt_eventDate_max', 'score_lifetime_annotation']:
#     del collectors_dwcagent_output[this_column]


In [26]:
show_display_output=True
if show_display_output:
    criterion = collectors_dwcagent_output['alternateName'].map(lambda x: x.startswith('S. Ahmad'))
    criterion = collectors_dwcagent_output['custom_score_multiple_names'].map(lambda x: x < 0 )
    
    display(collectors_dwcagent_output[criterion].head(20))

,occurrenceID,verbatimName,alternateName,displayOrder,name,attributionRemarks,startedAtTime,endedAtTime,agentType,action,agentIdentifierType,identifier,custom_score_overall,custom_namematch_similarity,custom_score_multiple_names,custom_score_lifetime
43,http://id.snsb.info/snsb/collection/444683/544...,"Ackermann, M. (no. 647)",M. Ackermann,1,Markus Ackermann,1.0 (cos similarity); 0.25 (score overall); 19...,2001-02-03,2022-10-06,Person,collected,wikidata,http://www.wikidata.org/entity/Q21504506,0.25,1.0,-0.5,1.0
44,http://id.snsb.info/snsb/collection/444683/544...,"Ackermann, M. (no. 647)",M. Ackermann,2,Marianne Ackermann,1.0 (cos similarity); -0.25 (score overall); ?...,2001-02-03,2022-10-06,Person,collected,wikidata,http://www.wikidata.org/entity/Q36619087,-0.25,1.0,-0.5,0.0
45,http://id.snsb.info/snsb/collection/444683/544...,"Ackermann, M. (no. 647)",M. Ackermann,3,Manfred Ackermann,1.0 (cos similarity); -0.25 (score overall); ?...,2001-02-03,2022-10-06,Person,collected,wikidata,http://www.wikidata.org/entity/Q47112660,-0.25,1.0,-0.5,0.0
90,https://herbarium.bgbm.org/object/B101191388,"Adler,W.",W. Adler,1,Wolfgang Adler,1.0 (cos similarity); 0.25 (score overall); 19...,1992-06-12,2020-06-28,Person,collected,wikidata,http://www.wikidata.org/entity/Q119062956,0.25,1.0,-0.5,1.0
89,https://herbarium.bgbm.org/object/B101191388,"Adler,W.",W. Adler,2,Wilhelm Adler,1.0 (cos similarity); -0.25 (score overall); ?...,1992-06-12,2020-06-28,Person,collected,wikidata,http://www.wikidata.org/entity/Q10393599,-0.25,1.0,-0.5,0.0
143,https://herbarium.bgbm.org/object/B101097885,"Ahmad,S.",S. Ahmad,1,Sultan Ahmad,1.0 (cos similarity); 0.25 (score overall); 19...,1934-09-01,1957-01-15,Person,collected,wikidata,http://www.wikidata.org/entity/Q47112765,0.25,1.0,-0.5,1.0
144,https://herbarium.bgbm.org/object/B101097885,"Ahmad,S.",S. Ahmad,2,Suhail Ahmad,1.0 (cos similarity); -0.25 (score overall); ?...,1934-09-01,1957-01-15,Person,collected,wikidata,http://www.wikidata.org/entity/Q88848655,-0.25,1.0,-0.5,0.0
142,https://herbarium.bgbm.org/object/B101097885,"Ahmad,S.",S. Ahmad,3,Shamsuddin Ahmad,1.0 (cos similarity); -0.50 (score overall); 1...,1934-09-01,1957-01-15,Person,collected,wikidata,http://www.wikidata.org/entity/Q33661104,-0.50,1.0,-0.5,-0.5
199,http://id.snsb.info/snsb/collection/459414/560...,"Albo, G.",G. Albo,1,Giacomo Albo,1.0 (cos similarity); 0.25 (score overall); 18...,1933-04-30,1933-04-30,Person,collected,wikidata,http://www.wikidata.org/entity/Q21505027,0.25,1.0,-0.5,1.0
198,http://id.snsb.info/snsb/collection/459414/560...,"Albo, G.",G. Albo,2,Giacomo Albo,1.0 (cos similarity); -0.25 (score overall); ?...,1933-04-30,1933-04-30,Person,collected,wikidata,http://www.wikidata.org/entity/Q10289422,-0.25,1.0,-0.5,0.0


In [27]:
# TODO further evaluation or filtering, counting, clean up aso.
if not os.path.exists('data'):
    os.makedirs('data')

# this_timestamp_for_data=time.strftime('%Y%m%d') # 20230719
this_timestamp_for_data=20230823
this_output_file='data/results_bgbm_collectors-eventDate_vs_wikidata-botanists_cossim-similarity_dwc-agent-output_%s.csv' % (
    this_timestamp_for_data
)

collectors_dwcagent_output.to_csv(this_output_file, index=False)

print("Wrote matches of collector names as dwc-agent-output into %s (%d kB)" % 
    (this_output_file, os.path.getsize(this_output_file) >> 10 ) # 10000 >> 10 = bitshift operator, to get kilo bytes (10-bits=>1024)
)

Wrote matches of collector names as dwc-agent-output into data/results_bgbm_collectors-eventDate_vs_wikidata-botanists_cossim-similarity_dwc-agent-output_20230823.csv (6265 kB)


## Documentation

TODO document columns

Explanation of columns:

Column | Description
-|-
**Botanical collectors** |
family | parsed family name
given | parsed given name
suffix | suffix from name parsing
particle | particle from name parsing
dropping_particle | dropping_particle from name parsing
nick | nick name from name parsing
appellation | appellation from name parsing
title | title from name parsing
occurrenceID_collectors_count | count of all occurrenceID of one particular collector name
occurrenceID_collectors_firstsample | a data sample of an occurrenceID 
eventDate | date of the sampling event (required by GBIF, ☞ https://www.gbif.org/data-quality-requirements-sampling-events)
eventDate_min | calculated earliest date of all the sampling events within the data
eventDate_max | calculated latest date of all the sampling events within the data
eventDate_mean | calculated mean date of all the sampling events within the data
TODO activity_span | Number of years between first and last collection
**Name matching** |
namematch_source_data | matched name of the collector data set
namematch_resource_data | matched name of Wikidata the collector was tried to matched to
namematch_similarity | calculated cosine-similarity
**DarwinCore Agent Output** | (☞ [agent_actions_v2020-09-08.xml](https://github.com/tdwg/attribution/blob/master/people/dwc/agent_actions_v2020-09-08.xml))
occurrenceID | occurrence ID of the data item
name | the interpreted name match (https://github.com/tdwg/attribution/ The name of the item. In this case the *full name* as would be written on a legal document (without abbreviation), eg givenName familyName)
verbatimName | the source data name(s) (https://github.com/tdwg/attribution/ As written on occurrence, such as the collection or determination label.)
alternateName | the input name, collector source name (An alias for the item. Other full name agent may have been known under such as maiden name.)
displayOrder | I guess ordering the multiple name cases (https://github.com/tdwg/attribution/ The display order for the agent that executed the action when more than one agent was a participant.)
attributionRemarks | notes on the results (distance or similarity), including calculated value
agentType | The nature of the agent, e.g. "Person", "Organization", "SoftwareApplication"
action | The name of the single action written as a verb in past tense. Recommended best practice is to use a controlled vocabulary, examples "collected" or "identified"
agentIdentifierType | The type of identifier for the agent. (https://github.com/tdwg/attribution/ Recommended best practice is to use a controlled vocabulary, e.g. “ORCID”, “ISNI”, “Wikidata”, “VIAF”, “RoR”, “Ringgold”, “GRID”).
identifier | Wikidata ID (Recommended practice is to identify the resource by means of a string conforming to an identification system. Examples include International Standard Book Number (ISBN), Digital Object Identifier (DOI), and Uniform Resource Name (URN). Persistent identifiers should be provided as HTTP URIs.)
startedAtTime | (https://github.com/tdwg/attribution/ Start is when an action is deemed to have been started by an agent.) the first date of eventDate (supposedly the first sampling date), but grouped from collector name—in case of multiple name matches this first “sampling date” is less reliable and be reliable to relate to the source collector’s life time.
endedAtTime | (https://github.com/tdwg/attribution/ End is when an action is deemed to have been ended by an agent.) the last date of eventDate (supposedly the last sampling date), but grouped from collector name—in case of multiple name matches this first “sampling date” is less reliable and be reliable to relate to the source collector’s life time.
**Wikidata** |
item | Wikidata Item ID (URL)
itemLabel | Wikidata Item label (perhaps similar to the full name)
surname	| Surname; derived from item label
initials | Initials; derived from item label
canonical_string | Canonical name string; derived from item label, used for matching
orcid | ORCID ([P496](https://www.wikidata.org/wiki/Property:P496))
viaf | VIAF ID ([P214](https://www.wikidata.org/wiki/Property:P214))
isni | ISNI ID ([P213](https://www.wikidata.org/wiki/Property:P496))	
harv | Harvard Index of Botanists ID ([P6264](https://www.wikidata.org/wiki/Property:P6264))
ipni | IPNI author ID ([P586](https://www.wikidata.org/wiki/Property:P586))
abbr | botanist author abbreviation (standard form) ([P428](https://www.wikidata.org/wiki/Property:P428))
bionomia_id | identifier for a collector and/or determiner of natural history specimens, in the Bionomia database ([P6944](https://www.wikidata.org/wiki/Property:P6944))
yob	| Year of birth (derived from [P569](https://www.wikidata.org/wiki/Property:P569))
yod	| Year of death (derived from [P496](https://www.wikidata.org/wiki/Property:P570))
wyb	| Work year period begin ([P2031](https://www.wikidata.org/wiki/Property:P2031))
wye | Work year period end ([P2032](https://www.wikidata.org/wiki/Property:P2032))